In [3]:
import sys
from typing import List, Dict, Iterable, Tuple

In [28]:
transform = "nnbaa$"
table = ['' for _ in transform]
    # Step 2: Iteratively reconstruct each column of the original sorted rotations table
for i in range(len(transform)):
    # Prepend the BWT string to each entry (building the table column by column)
    table = [transform[j] + table[j] for j in range(len(transform))]
    print('pre',table)
    # Sort the table based on the newly prepended characters
    
    table.sort()
    print(table)

#return table[0]

pre ['n', 'n', 'b', 'a', 'a', '$']
['$', 'a', 'a', 'b', 'n', 'n']
pre ['n$', 'na', 'ba', 'ab', 'an', '$n']
['$n', 'ab', 'an', 'ba', 'n$', 'na']
pre ['n$n', 'nab', 'ban', 'aba', 'an$', '$na']
['$na', 'aba', 'an$', 'ban', 'n$n', 'nab']
pre ['n$na', 'naba', 'ban$', 'aban', 'an$n', '$nab']
['$nab', 'aban', 'an$n', 'ban$', 'n$na', 'naba']
pre ['n$nab', 'naban', 'ban$n', 'aban$', 'an$na', '$naba']
['$naba', 'aban$', 'an$na', 'ban$n', 'n$nab', 'naban']
pre ['n$naba', 'naban$', 'ban$na', 'aban$n', 'an$nab', '$naban']
['$naban', 'aban$n', 'an$nab', 'ban$na', 'n$naba', 'naban$']


In [211]:
def inverse_burrows_wheeler_transform(transform: str) -> str:
    table = ['' for _ in transform]
    for i in range(len(transform)):
        table = [transform[j] + table[j] for j in range(len(transform))]
        table.sort()
    return table[0][1:]+"$"

In [212]:
inverse_burrows_wheeler_transform("TTCCTAACG$A")

'TACATCACGT$'

In [72]:
for i,char in enumerate('bat'):
    print(i,char)

0 b
1 a
2 t


In [109]:
def last2first(bwt:str)->List[int]:
    first_col = sorted(bwt)
    occur_ind_dict = {} #dict with key=symbol,val=array with ind=nth occurence(subscript) & val=ind of occur in first_col
    count_so_far_dict = {}
    l2f = [0 for _ in range(len(bwt))]
    for i,char in enumerate(first_col):
        if char not in occur_ind_dict:
            occur_ind_dict[char]=[i]
        else:
            occur_ind_dict[char].append(i)
    for i,char in enumerate(bwt):
        if char not in count_so_far_dict:
            count_so_far_dict[char]=0
        else:
            count_so_far_dict[char]+=1
        ind = occur_ind_dict[char][count_so_far_dict[char]]
        l2f[i]=ind
    return l2f 

In [111]:
last2first('smnpbnnaaaaa$a')

[13, 8, 9, 12, 7, 10, 11, 1, 2, 3, 4, 5, 0, 6]

In [106]:
def bw_match_l2f(bwt:str,pattern:str,l2f:List[int])->int: #returns num occurences of pattern
    bwt_len = len(bwt)
    top = 0
    bottom = bwt_len-1
    while top<=bottom:
        if pattern:
            symbol = pattern[-1]
            pattern = pattern[:-1]
            topIndex = bwt[top:bottom+1].find(symbol)
            if topIndex!=-1:
                topIndex+=top 
                bottomIndex = bwt[top:bottom+1].rfind(symbol)+top
                #print(bottomIndex)
                top = l2f[topIndex]
                bottom = l2f[bottomIndex]
            else:
                return 0
        else:
            return bottom-top+1

In [107]:
def bw_matching(bwt: str, patterns: List[str]) -> List[int]:
    output = []
    l2f = last2first(bwt)
    for pattern in patterns:
        output.append(bw_match_l2f(bwt,pattern,l2f))
    return output

In [110]:
bw_matching("TCCTCTATGAGATCCTATTCTATGAAACCTTCA$GACCAAAATTCTCCGGC",["CCT","CAC","GAG","CAG","ATC"])

[2, 1, 1, 0, 1]

better BW Matching

In [74]:
def first_occur(first_col:str)->Dict:
    first_occur_dict = {}
    for i,char in enumerate(first_col):
        if char not in first_occur_dict:
            first_occur_dict[char]=i
    return first_occur_dict

In [77]:
first_occur("$aaaaaabmnnnps")

{'$': 0, 'a': 1, 'b': 7, 'm': 8, 'n': 9, 'p': 12, 's': 13}

In [86]:
def bw_count(bwt:str):
    count_dict = {}
    first_col = sorted(bwt)
    first_occur_dict = first_occur(first_col)
    for key in first_occur_dict:
        count_dict[key]=[0]
    for char in bwt:
        for key in count_dict:
            if key==char:
                curr_count = count_dict[char][-1]
                count_dict[char].append(curr_count+1)
            else:
                curr_count = count_dict[key][-1]
                count_dict[key].append(curr_count)   
    return first_occur_dict, count_dict

In [87]:
bw_count("smnpbnnaaaaa$a")

({'$': 0, 'a': 1, 'b': 7, 'm': 8, 'n': 9, 'p': 12, 's': 13},
 {'$': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
  'a': [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 5, 6],
  'b': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'm': [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'n': [0, 0, 0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3],
  'p': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  's': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [88]:
def better_bw_matching_pattern(first_occur_dict:Dict,bwt:str,pattern:str,count_dict:Dict):
    top = 0
    bottom = len(bwt)-1
    while top<=bottom:
        if pattern:
            symbol = pattern[-1]
            pattern = pattern[:-1]
            if bwt[top:bottom+1].find(symbol)!=-1:
                top = first_occur_dict[symbol]+count_dict[symbol][top]
                bottom = first_occur_dict[symbol]+count_dict[symbol][bottom+1]-1
            else:
                return 0
        else:
            return bottom-top+1

In [91]:
bwt = "smnpbnnaaaaa$a"
first_occur_dict,count_dict = bw_count(bwt)
better_bw_matching_pattern(first_occur_dict,bwt,'ba',count_dict)

1

In [92]:
def better_bw_matching(bwt: str, patterns: List[str]) -> List[int]:
    first_occur_dict,count_dict = bw_count(bwt)
    pattern_count = []
    for pattern in patterns:
        count = better_bw_matching_pattern(first_occur_dict,bwt,pattern,count_dict)
        pattern_count.append(count)
    
    return pattern_count

In [93]:
better_bw_matching("GGCGCCGC$TAGTCACACACGCCGTA",["ACC","CCG","CAG"])

[1, 2, 1]

In [165]:
def return_pos_parr(bwt:str,ind:int,parr:List[Tuple[int,int]],l2f:List[int],K:int): #returns val of suffix array given index of bwt
    iter_count = 0  
    parr_ind = [K_ind for (pos,K_ind) in parr if pos==ind]
    if parr_ind:
        return parr_ind[0]
    else:
        while not parr_ind:
            ind = l2f[ind]
            iter_count+=1
            parr_ind = [K_ind for (pos,K_ind) in parr if pos==ind]
        return return_pos_parr(bwt,ind,parr,l2f,K)+iter_count

In [189]:
def return_count(symbol:str,K:int,checkpoint_arr,bwt:str,ptr:int,first_occur_dict):
    start = K*(ptr//K)
    if first_occur_dict[symbol]>start:
        start = first_occur_dict[symbol]
    return checkpoint_arr[symbol][ptr//K]+sum([1 if char == symbol else 0 for char in bwt[start:ptr+1]])

In [250]:
def pattern_matching_partial(bwt:str,pattern:str,checkpoint_arr:Dict,first_occur_dict:Dict,parr:List[Tuple[int, int]],K:int,text:str,l2f:List[int])->int:
    top = 0
    bottom = len(bwt)-1
    while top<=bottom:
        if pattern:
            symbol = pattern[-1]
            pattern = pattern[:-1]
            if bwt[top:bottom+1].find(symbol)!=-1: 
                
                print("p",checkpoint_arr[symbol][top//K],sum([1 if char == symbol else 0 for char in bwt[K*(top//K):top]]),first_occur_dict[symbol])
                top_count = checkpoint_arr[symbol][(top//K)]+sum([1 if char==symbol else 0 for char in bwt[K*(top//K):top]])
                top = first_occur_dict[symbol]+top_count
                bottom_count = checkpoint_arr[symbol][(bottom+1)//K]+sum([1 if char==symbol else 0 for char in bwt[K*((bottom+1)//K):bottom+1]])
                if top==11 and bottom==14:
                    print('special')
                    print(bottom_count,first_occur_dict[symbol])
                bottom = first_occur_dict[symbol]+bottom_count-1
                
                print(top,bottom)
            else:
                return 0
        else:
            print('top',top,'bottom',bottom)
            pattern_ind = []
            print(inverse_burrows_wheeler_transform(bwt))
            for i in range(top,bottom+1):
                pattern_ind.append(return_pos_parr(bwt,i,parr,l2f,K))
            return pattern_ind
            


In [243]:
14//5

2

In [223]:
def burrows_wheeler_transform(text: str) -> str:
    sarr = [i for i in range(len(text))]
    sarr.sort(key=lambda item: text[item:]+text[:item])
    #for i in sarr:
        #print(text[i:]+text[:i],text[i-1])
    bwtarr = ''.join([text[i-1] for i in sarr])
    return bwtarr

In [236]:
def multiple_pattern_matching(text: str, patterns: List[str]) -> Dict[str, List[int]]:
    """
    Find all starting positions in text where each string from patterns appears as a substring, allowing for up to d mismatches.
    """
    text = text+"$"
    K= 5
    bwt = burrows_wheeler_transform(text) 
    print(bwt)
    l2f = last2first(bwt)
    print(l2f)
    first_occur_dict,checkpoint_arr = bw_checkpoint_array(bwt,K)
    parr = partial_suffix_array(text,K)
    pattern_dict = {}
    for pattern in patterns:
        pattern_dict[pattern]= pattern_matching_partial(bwt,pattern,checkpoint_arr,first_occur_dict,parr,K,text,l2f)
    return pattern_dict

In [227]:
print(burrows_wheeler_transform("AATCGGGTTCAATCGGGGT"))

CTAATTTCGCGGGGGTAAG


In [254]:
multiple_pattern_matching("ATATATATAT",["GT","AGCT","TAA","AAT","AATAT"])

TTTTTAAAAA
[5, 6, 7, 8, 9, 0, 1, 2, 3, 4]
p 0 0 5
5 9
p 0 0 5
5 9
p 0 0 0
0 4
p 0 0 5
5 9
p 0 0 0
0 4
p 0 0 5
5 9
p 0 0 0
0 4
p 0 0 5
5 9
p 0 0 0
0 4


{'GT': 0, 'AGCT': 0, 'TAA': 0, 'AAT': 0, 'AATAT': 0}

In [252]:
multiple_pattern_matching("AATCGGGTTCAATCGGGGT$",["ATCG","GGGT"])

TC$AATTTCGCGGGGGTAAG
[15, 5, 0, 1, 2, 16, 17, 18, 6, 8, 7, 9, 10, 11, 12, 13, 19, 3, 4, 14]
p 0 0 8
8 14
p 1 0 5
6 7
p 1 1 15
17 18
p 2 0 1
3 4
top 3 bottom 4
AATCGGGTTCAATCGGGGT$
p 0 0 15
15 19
p 5 0 8
13 14
p 1 2 8
special
5 8
11 12
p 1 0 8
9 10
top 9 bottom 10
AATCGGGTTCAATCGGGGT$


{'ATCG': [11, 1], 'GGGT': [15, 4]}

In [216]:
bwt = "smnpbnnaaaaa$a"
pattern = 'ana'
K=5
first_occur_dict,checkpoint_arr= bw_checkpoint_array(bwt,K)
parr=[]
pattern_matching_partial(bwt,pattern,checkpoint_arr,first_occur_dict,parr,K)

p 0 0 1
1 6
p 0 0 9
9 11
p 0 3 1
3 5
top 3 bottom 5
panamabananas$


[1, 7, 9]

In [144]:
def bw_checkpoint_array(bwt:str,K:int):
    partial_count_dict = {}
    first_col = sorted(bwt)
    first_occur_dict = first_occur(first_col)
    for key in first_occur_dict:
        partial_count_dict[key]=[0]
    for i,char in enumerate(bwt):
        #print(i,i%K,char)
        for key in partial_count_dict:
            if key==char:
                curr_count = partial_count_dict[key][-1] +1
            else:
                curr_count = partial_count_dict[key][-1]
            #print(char,key,curr_count)
            if i>=((len(bwt)//K)*K):
                return first_occur_dict,partial_count_dict
            if i%K==0:
                partial_count_dict[key].append(curr_count)
            else:
                partial_count_dict[key][-1]=curr_count
            
    return first_occur_dict,partial_count_dict

In [145]:
bw_checkpoint_array("smnpbnnaaaaa$a",5)

({'$': 0, 'a': 1, 'b': 7, 'm': 8, 'n': 9, 'p': 12, 's': 13},
 {'$': [0, 0, 0],
  'a': [0, 0, 3],
  'b': [0, 1, 1],
  'm': [0, 1, 1],
  'n': [0, 1, 3],
  'p': [0, 1, 1],
  's': [0, 1, 1]})

In [ ]:
def multiple_pattern_matching(text: str, patterns: List[str]) -> Dict[str, List[int]]:
    """
    Find all starting positions in text where each string from patterns appears as a substring.
    """
    #make bwt
    
    for pattern in patterns:


In [10]:
def partial_suffix_array(text: str, k: int) -> List[Tuple[int, int]]:
    """
    Generate a partial suffix array for the given text and interval K.
    """
    sarr = suffix_array(text)
    #print(sarr)
    psarr = [(i,sarr[i]) for i in range(len(sarr)) if sarr[i]%k==0]
    #print(psarr)
    return psarr

def suffix_array(text: str) -> List[int]:
    sarr = [i for i in range(len(text))]
    sarr.sort(key=lambda item: text[item:])
    return sarr

In [208]:
partial_suffix_array("panamabananas$",5)

[(1, 5), (11, 10), (12, 0)]